# Telecom Traffic Forecasting with TensorFlow GradientTape  
*A practical demonstration of backpropagation & autograd in a custom training loop.*

This notebook builds a small but realistic telecom-style forecasting experiment using TensorFlow 2.x.  
The focus is on:

- synthetic yet meaningful traffic data  
- multi-feature regression (hour, weekday, previous traffic)  
- a DNN trained with a **manually written training loop**  
- explicit backprop via **`tf.GradientTape()`**  
- metrics & validation splits

# 1. Imports & Seeds

We keep everything deterministic to make training runs reproducible.

In [1]:
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

# 2. Synthetic Telecom Data  
Real-world telecom traffic usually shows:

- daily periodicity (rush hours)
- weekly periodicity (weekday vs weekend)
- lag dependence (previous hour influences current hour)
- noise from unpredictable events

We simulate **120 days of hourly data** and engineer three input features:

**Features**
- hour_of_day (0–23)
- day_of_week (0–6)
- prev_hour_traffic (lag 1)

**Target**
- current hour traffic

In [2]:
num_days = 120
hours_per_day = 24
n_samples = num_days * hours_per_day

hours = np.tile(np.arange(hours_per_day), num_days)
days = np.repeat(np.arange(num_days) % 7, hours_per_day)

# Seasonality patterns
daily_pattern = 10 + 5 * np.sin(2 * np.pi * hours / 24.0)
weekly_pattern = 2 * np.sin(2 * np.pi * days / 7.0)

noise = np.random.normal(scale=1.0, size=n_samples)

traffic = daily_pattern + weekly_pattern + noise

prev_traffic = np.concatenate([[traffic[0]], traffic[:-1]])

# Stack features
X = np.stack(
    [
        hours / 23.0,
        days / 6.0,
        prev_traffic / 20.0
    ],
    axis=1
).astype(np.float32)

y = traffic.astype(np.float32).reshape(-1, 1)


# 3. Train–Test Split and Data Pipeline  
We use an 80/20 split and build a `tf.data.Dataset` pipeline for efficient batching.

In [3]:
split = int(0.8 * n_samples)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

batch_size = 64

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(1000)
    .batch(batch_size)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
)


# 4. Model Definition  
A lightweight DNN:

Input(3) → Dense(32, relu) → Dense(16, relu) → Dense(1)


This keeps the architecture simple but expressive enough for a seasonal time-series regression task.


In [4]:
def build_model():
    return tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(3,)),
            tf.keras.layers.Dense(32, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu"),
            tf.keras.layers.Dense(1)
        ]
    )

model = build_model()
optimizer = tf.keras.optimizers.Adam(1e-3)
loss_fn = tf.keras.losses.MeanSquaredError()

train_mae = tf.keras.metrics.MeanAbsoluteError(name="train_mae")
val_mae = tf.keras.metrics.MeanAbsoluteError(name="val_mae")


# 5. Custom Training Step (Backprop via GradientTape)  
This is the key part of the project.

We do **not** use `model.fit()`.

Instead, we explicitly:

1. open a `tf.GradientTape()`  
2. run the forward pass  
3. compute the loss  
4. backpropagate with `tape.gradient()`  
5. update parameters with `optimizer.apply_gradients()`

This mirrors exactly what *backpropagation* does inside TensorFlow’s autograd engine.


In [5]:
@tf.function
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        preds = model(x_batch, training=True)
        loss = loss_fn(y_batch, preds)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_mae.update_state(y_batch, preds)
    return loss


@tf.function
def val_step(x_batch, y_batch):
    preds = model(x_batch, training=False)
    loss = loss_fn(y_batch, preds)
    val_mae.update_state(y_batch, preds)
    return loss


# 6. Training Loop  
A production-style loop:

- resets metrics per epoch  
- accumulates batch losses  
- prints validation metrics  
- no Keras shortcuts  

This shows understanding of how *GradientTape enables backprop* in TensorFlow.


In [6]:
epochs = 25

for epoch in range(1, epochs + 1):
    train_mae.reset_state()
    val_mae.reset_state()

    train_losses = []
    for xb, yb in train_ds:
        loss = train_step(xb, yb)
        train_losses.append(loss)

    val_losses = []
    for xb, yb in test_ds:
        vloss = val_step(xb, yb)
        val_losses.append(vloss)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={tf.reduce_mean(train_losses):.4f} "
        f"val_loss={tf.reduce_mean(val_losses):.4f} "
        f"train_MAE={train_mae.result():.3f} "
        f"val_MAE={val_mae.result():.3f}"
    )


Epoch 01 | train_loss=106.8639 val_loss=94.3992 train_MAE=9.579 val_MAE=8.880
Epoch 02 | train_loss=85.8152 val_loss=69.3079 train_MAE=8.389 val_MAE=7.327
Epoch 03 | train_loss=56.7871 val_loss=38.9817 train_MAE=6.475 val_MAE=5.147
Epoch 04 | train_loss=28.7186 val_loss=19.7406 train_MAE=4.385 val_MAE=3.717
Epoch 05 | train_loss=16.7523 val_loss=15.2202 train_MAE=3.411 val_MAE=3.268
Epoch 06 | train_loss=13.2755 val_loss=12.1585 train_MAE=3.042 val_MAE=2.909
Epoch 07 | train_loss=10.4924 val_loss=9.4509 train_MAE=2.691 val_MAE=2.556
Epoch 08 | train_loss=8.1055 val_loss=7.2502 train_MAE=2.355 val_MAE=2.224
Epoch 09 | train_loss=6.1891 val_loss=5.5290 train_MAE=2.045 val_MAE=1.924
Epoch 10 | train_loss=4.7496 val_loss=4.2743 train_MAE=1.779 val_MAE=1.680
Epoch 11 | train_loss=3.7678 val_loss=3.4512 train_MAE=1.579 val_MAE=1.507
Epoch 12 | train_loss=3.1405 val_loss=2.9239 train_MAE=1.440 val_MAE=1.389
Epoch 13 | train_loss=2.7613 val_loss=2.6156 train_MAE=1.351 val_MAE=1.314
Epoch 14 | 

# 7. Sample Predictions  
A quick qualitative check to see if the model learned the shape of the traffic pattern.


In [7]:
sample = X_test[:5]
pred = model(sample, training=False)

print("\nSample predictions (true vs pred):")
for true_y, pred_y in zip(y_test[:5], pred.numpy()):
    print(f"true={true_y[0]:6.2f}  pred={pred_y[0]:6.2f}")



Sample predictions (true vs pred):
true= 10.05  pred= 10.26
true= 12.45  pred= 11.44
true= 11.16  pred= 12.95
true= 11.40  pred= 11.81
true= 12.91  pred= 11.77


# Notebook Summary

The key objective was to demonstrate **backpropagation and gradient flow** through a realistic regression model using:

- TensorFlow's autograd system (`GradientTape`)
- custom training & validation loops
- synthetic telecom-style traffic data
- meaningful feature engineering
- proper metrics and validation handling
